In [1]:
%cd /opt

/opt


In [2]:
%%capture
!tar xvf /kaggle/input/extract-prebuilt-kaldi-from-docker/kaldi.tar

In [3]:
%cd kaldi/egs

/opt/kaldi/egs


In [4]:
!git clone https://github.com/danijel3/ClarinStudioKaldi

Cloning into 'ClarinStudioKaldi'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 778 (delta 0), reused 0 (delta 0), pack-reused 775
Receiving objects: 100% (778/778), 35.26 MiB | 19.50 MiB/s, done.
Resolving deltas: 100% (262/262), done.


In [5]:
%cd ClarinStudioKaldi

/opt/kaldi/egs/ClarinStudioKaldi


In [6]:
#apt-get -y install libperlio-gzip-perl
!conda install -c bioconda perl-perlio-gzip -y

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

In [7]:
import os
#os.environ['LD_LIBRARY_PATH'] = f'{os.environ["LD_LIBRARY_PATH"]}:/opt/kaldi/tools/openfst-1.6.7/lib:/opt/kaldi/src/lib'
os.environ['LD_LIBRARY_PATH'] = '/opt/conda/lib:/opt/kaldi/tools/openfst-1.6.7/lib:/opt/kaldi/src/lib'

In [8]:
!cat path.sh|sed -e 's/~\/apps/\/opt/' > tmp
!mv tmp path.sh

In [9]:
!echo > local_clarin/clarin_pl_clean.sh

In [10]:
!ln -s ../wsj/s5/steps
!ln -s ../wsj/s5/conf
!ln -s ../wsj/s5/local
!ln -s ../wsj/s5/utils

In [11]:
!cp -r /kaggle/input/kaldi-clarinstudio-polish-data-prep/data /kaggle/working/

In [12]:
!mkdir /kaggle/working/exp
!ln -s /kaggle/working/exp
!ln -s /kaggle/working/data

In [13]:
%%writefile run.sh
. path.sh

export nj=40 ##number of concurrent processes
export nj_test=30 ## number of concurrent processes for test has to be <=30

#train Monophone system
steps/train_mono.sh --nj $nj data/train data/lang_nosp exp/mono0

#align using the Monophone system
steps/align_si.sh --nj $nj data/train data/lang_nosp exp/mono0 exp/mono0_ali

#train initial Triphone system
steps/train_deltas.sh 2000 10000 data/train data/lang_nosp exp/mono0_ali exp/tri1

#re-align using the initial Triphone system
steps/align_si.sh --nj $nj data/train data/lang_nosp exp/tri1 exp/tri1_ali

#train tri2a, which is deltas + delta-deltas
steps/train_deltas.sh 2500 15000 data/train data/lang_nosp exp/tri1_ali exp/tri2a

#train tri2b, which is tri2a + LDA
steps/train_lda_mllt.sh --splice-opts "--left-context=3 --right-context=3" \
   2500 15000 data/train data/lang_nosp exp/tri1_ali exp/tri2b

#re-align tri2b system
steps/align_si.sh --nj $nj --use-graphs true data/train data/lang_nosp exp/tri2b exp/tri2b_ali 


#from 2b system, train 3b which is LDA + MLLT + SAT.
steps/train_sat.sh 2500 15000 data/train data/lang_nosp exp/tri2b_ali exp/tri3b

#get pronounciation probabilities and silence information 
./steps/get_prons.sh data/train data/lang_nosp exp/tri3b

#recreate dict with new pronounciation and silence probabilities
./utils/dict_dir_add_pronprobs.sh data/local/dict_nosp \
	exp/tri3b/pron_counts_nowb.txt \
	exp/tri3b/sil_counts_nowb.txt \
	exp/tri3b/pron_bigram_counts_nowb.txt data/local/dict

#recreate lang directory
utils/prepare_lang.sh data/local/dict "<unk>" data/local/tmp data/lang

#recreate G.fst
utils/format_lm.sh data/lang local_clarin/arpa.lm.gz data/local/dict/lexicon.txt data/lang_test

#download a large LM (~843MB)
if [ ! -f local_clarin/large.arpa.gz ] ; then
(
	cd local_clarin
	curl -O http://mowa.clarin-pl.eu/korpusy/large.arpa.gz
)
fi

#create the const-arpa lang dir
./utils/build_const_arpa_lm.sh local_clarin/large.arpa.gz data/lang data/lang_carpa

#from 3b system, align all data
steps/align_fmllr.sh --nj $nj data/train data/lang exp/tri3b exp/tri3b_ali

#train MMI on tri3b (LDA+MLLT+SAT)
steps/make_denlats.sh --nj $nj --transform-dir exp/tri3b_ali data/train data/lang \
	exp/tri3b exp/tri3b_denlats
steps/train_mmi.sh data/train data/lang exp/tri3b_ali exp/tri3b_denlats exp/tri3b_mmi

#test Monophone system
utils/mkgraph.sh data/lang_nosp_test exp/mono0 exp/mono0/graph
steps/decode.sh --nj $nj_test exp/mono0/graph data/test exp/mono0/decode

#test initial Triphone system
utils/mkgraph.sh data/lang_nosp_test exp/tri1 exp/tri1/graph
steps/decode.sh --nj $nj_test exp/tri1/graph data/test exp/tri1/decode

#test tri2a
utils/mkgraph.sh data/lang_nosp_test exp/tri2a exp/tri2a/graph
steps/decode.sh --nj $nj_test exp/tri2a/graph data/test exp/tri2a/decode

#test tri2b
utils/mkgraph.sh data/lang_nosp_test exp/tri2b exp/tri2b/graph
steps/decode.sh --nj $nj_test exp/tri2b/graph data/test exp/tri2b/decode

#test tri3b
utils/mkgraph.sh data/lang_nosp_test exp/tri3b exp/tri3b/graph_nosp
steps/decode_fmllr.sh --nj $nj_test exp/tri3b/graph_nosp data/test exp/tri3b/decode_nosp

#test tri3b again
utils/mkgraph.sh data/lang_test exp/tri3b exp/tri3b/graph
steps/decode_fmllr.sh --nj $nj_test exp/tri3b/graph data/test exp/tri3b/decode

Overwriting run.sh


In [14]:
!bash run.sh

steps/train_mono.sh --nj 40 data/train data/lang_nosp exp/mono0
steps/train_mono.sh: Initializing monophone system.
steps/train_mono.sh: Compiling training graphs
steps/train_mono.sh: Aligning data equally (pass 0)
steps/train_mono.sh: Pass 1
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 2
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 3
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 4
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 5
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 6
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 7
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 8
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 9
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 10
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 11
steps/train_mono.sh: Pass 12
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 13
steps/train_mono.sh: Pass 14
ste

In [15]:
!find exp -type l|zip /kaggle/working/links.zip -@

  adding: exp/ (stored 0%)
